In [6]:
# This module will annotate the primers and guides on original 
# DNA sequence. The reverse primers/guides will show the direction when opened
# in SnapGene, but forward oligos won't show directionality. due to some 
# bug in converting genbank to .dna file using SnapGene.

# The .dna (SnapGene format) to genbank and then again to .dna conversion using python is 
# buggy and generates errors. So, the .dna to genbank conversion needs 
# to be done manually using SnapGene. 
# The output will be genbank file in a subfolder "Annotated_Sequences". 




from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.SeqFeature import CompoundLocation
from Bio.Seq import Seq
from natsort import natsorted
import pandas as pd
import glob
import os


def write_annotated_seq(Combined_Seq, plasmid_name):
        
    curr_dir = os.getcwd()
    subdirectory = 'Annotated_Sequences'
    new_dir = os.path.join(curr_dir,subdirectory)
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        SeqIO.write(Combined_Seq, plasmid_name, "genbank")
        print(' File written:   \t', plasmid_name, '\n')
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        SeqIO.write(Combined_Seq, plasmid_name, "genbank")
        print('\n File written:   \t', plasmid_name, '\n')
        os.chdir('..')
        

def read_genbank_filename():
    
    DNA_files = natsorted( glob.glob("*.gb"))
    
    print('Files read:- \n ')
    print('Total files processed:  ', len(DNA_files))

    for filename in DNA_files:
        print('\t',filename)
    print('\n')
    
    for filename in DNA_files:
        CSV_file = 'Primers_Guides_neb_'+filename[:-3]+'.csv'        
        Read_genbank_file(filename, CSV_file)
        
        
def Read_genbank_file(filename, CSV_file):
    
    new_filename = filename[:-3]+'_annotated.gb'
    primer_guide_df = pd.read_csv(CSV_file)       
    Fragment_list = []
    
    for gb_record in SeqIO.parse(open(filename,"r"), "genbank") :
        
        DNA_len = len(gb_record.seq)
        New_seq = gb_record.seq + gb_record.seq

        for index, row in primer_guide_df.iterrows():
            Sequence = primer_guide_df['Sequence'].loc[index]
            Name = primer_guide_df['Name'].loc[index]
            newName = Find_label(Name)
            start_loc, end_loc, strand_val = Find_location(Sequence, New_seq, DNA_len)

            if end_loc > DNA_len:
                Loc1 = FeatureLocation(start_loc+1, DNA_len)
                Loc2 = FeatureLocation(0, (end_loc - DNA_len))
                if strand_val == 1:
                    f2 = SeqFeature(CompoundLocation([Loc1, Loc2]), type = 'misc_feature', strand = +1)
                else:
                    f2 = SeqFeature(CompoundLocation([Loc2, Loc1]), type = 'misc_feature', strand = strand_val)

                f2.qualifiers['label'] = newName
                gb_record.features.append(f2)
                
            else:
                if strand_val == 1:
                    f1 = SeqFeature(FeatureLocation(start_loc, end_loc), type = 'misc_feature', strand = +1)
                else:
                    f1 = SeqFeature(FeatureLocation(start_loc, end_loc), type = 'misc_feature', strand = strand_val)

                f1.qualifiers['label'] = newName
                gb_record.features.append(f1)            
            
        if index == len(primer_guide_df)-1:
            print('All primers and guides have been annotated')
        else:
            print('Problems with annotations')

        for features in (gb_record.features):
            if 'label' in features.qualifiers:
                Val = features.qualifiers['label']
                if 'fragment' in Val[0]:
                    #print(Val)
                    Fragment_list.append(Val[0])
                            
        write_annotated_seq(gb_record, new_filename)

                    
def Find_label(Name):

    if '_FP_' or '_EFP_' in Name:
        newName = Name.replace('FP', 'Fwd_primer')
    elif '_RP_' or '_ERP_' in Name:
        newName = Name.replace('RP', 'Rev_primer')

    elif '_G' and '_j' in Name:
        newName = Name.replace('_G', '_Guide_')
        
    return newName

        
def Find_location(Sequence, New_seq, DNA_len):
    
    DNA_Sequence = Seq(Sequence)
    strand_val = 1
    Location_1 = New_seq.find(DNA_Sequence)

    if Location_1 == -1:
        Sequence_revC = DNA_Sequence.reverse_complement()
        Location_1 = New_seq.find(Sequence_revC)
        strand_val = -1
    
    Location_2 = Location_1 + len(Sequence)
    
    start_loc = Location_1
    end_loc = Location_2
    # print(Sequence, start_loc, end_loc, strand_val)
    
    return start_loc, end_loc, strand_val
    
        
if __name__ == '__main__':
    
    print(os.getcwd(), '\n')
    read_genbank_filename()
    
    

/Users/nilmani/Desktop/Python/Plasmid_Annotation/Mammalian_II 

Files read:- 
 
Total files processed:   9
	 MZ11.gb
	 MZ12.gb
	 MZ13.gb
	 MZ14.gb
	 MZ15.gb
	 MZ16.gb
	 MZ17.gb
	 MZ19.gb
	 MZ20.gb


All primers and guides have been annotated
 File written:   	 MZ11_annotated.gb 

All primers and guides have been annotated
 File written:   	 MZ12_annotated.gb 

All primers and guides have been annotated
 File written:   	 MZ13_annotated.gb 

All primers and guides have been annotated
 File written:   	 MZ14_annotated.gb 

All primers and guides have been annotated
 File written:   	 MZ15_annotated.gb 

All primers and guides have been annotated
 File written:   	 MZ16_annotated.gb 

All primers and guides have been annotated
 File written:   	 MZ17_annotated.gb 

All primers and guides have been annotated
 File written:   	 MZ19_annotated.gb 

All primers and guides have been annotated
 File written:   	 MZ20_annotated.gb 

